<a href="https://colab.research.google.com/github/Cyber-Aju/MusicMe/blob/main/musicme_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# pip install tensorflow keras pickle nltk
#from google.colab import drive
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
#from tensorflow.keras.optimizers import SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']

intents_file = open('/content/drive/MyDrive/Colab Notebooks/musicme/intents.json').read()
intents = json.loads(intents_file)


import nltk
nltk.download('punkt')

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)


[(['hi'], 'greeting'), (['hello'], 'greeting'), (['whats', 'up'], 'greeting'), (['sup'], 'greeting'), (['is', 'anyone', 'there'], 'greeting'), (['whats', 'good'], 'greeting'), (['hey'], 'greeting'), (['bye'], 'goodbye'), (['cya'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['goodbye'], 'goodbye'), (['im', 'leaving'], 'goodbye'), (['have', 'a', 'good', 'day'], 'goodbye'), (['how', 'old', 'are', 'you'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['thanks'], 'thanks'), (['thank', 'you'], 'thanks'), (['thankyou'], 'thanks'), (['ty'], 'thanks'), (['I', 'owe', 'you', 'one'], 'thanks'), (['whats', 'is', 'your', 'name'], 'name'), (['whats', 'your', 'name'], 'name'), (['whats', 'should', 'I', 'call', 'you'], 'name'), (['how', 'should', 'I', 'address', 'you'], 'name'), (['Yes', 'it', 'does'], 'sky_net_yes'), (['Yeah'], 'sky_net_yes'), (['Haha', 'yep'], 'sky_net_yes'), (['yes'], 'sky_net_yes'), (['Indeed'], 'sky_net_yes'), (['Yup'], 'sky_net_yes'), (['Just', 'like', 'the', 'terminat

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import nltk
nltk.download('wordnet')
# create the training data
training = []
# create empty array for the output
output_empty = [0] * len(classes)

# training set, bag of words for every sentence
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    word_patterns = doc[0]
    # lemmatize each word - create a base word to represent related words
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # create the bag of words array with 1 if the word is found in the current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    # output is '0' for each tag and '1' for the current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle the features and create a NumPy array
random.shuffle(training)
training = np.array(training, dtype=object)  # Specify dtype as object

# create training and testing lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data is created")

Training data is created


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import tensorflow as tf
# Deep neural network model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compiling model. SGD with Nesterov accelerated gradient gives good results for this model
sgd = tf.keras.optimizers.legacy.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Training and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("Model is created")


Epoch 1/200
20/20 [==============================] - 1s 2ms/step - loss: 3.0885 - accuracy: 0.0303
Epoch 2/200
20/20 [==============================] - 0s 2ms/step - loss: 2.9645 - accuracy: 0.1010
Epoch 3/200
20/20 [==============================] - 0s 2ms/step - loss: 2.7675 - accuracy: 0.1818
Epoch 4/200
20/20 [==============================] - 0s 3ms/step - loss: 2.7509 - accuracy: 0.2525
Epoch 5/200
20/20 [==============================] - 0s 2ms/step - loss: 2.5574 - accuracy: 0.2626
Epoch 6/200
20/20 [==============================] - 0s 2ms/step - loss: 2.6291 - accuracy: 0.2424
Epoch 7/200
20/20 [==============================] - 0s 2ms/step - loss: 2.4469 - accuracy: 0.2626
Epoch 8/200
20/20 [==============================] - 0s 2ms/step - loss: 2.3426 - accuracy: 0.3232
Epoch 9/200
20/20 [==============================] - 0s 2ms/step - loss: 2.2726 - accuracy: 0.2929
Epoch 10/200
20/20 [==============================] - 0s 2ms/step - loss: 2.2738 - accuracy: 0.2525
Epoch 11/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('/content/drive/MyDrive/Colab Notebooks/musicme/intents.json').read())
words = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/musicme/words.pkl','rb'))
classes = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/musicme/classes.pkl','rb'))
def clean_up_sentence(sentence):
    # tokenize the pattern - splitting words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stemming every word - reducing to base form
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for words that exist in sentence
def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # bag of words - vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)
    return(np.array(bag))
def predict_class(sentence):
    # filter below  threshold predictions
    p = bag_of_words(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sorting strength probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
#Creating tkinter GUI
# import tkinter
# from tkinter import *
# def send():
#     msg = EntryBox.get("1.0",'end-1c').strip()
#     EntryBox.delete("0.0",END)
#     if msg != '':
#         ChatBox.config(state=NORMAL)
#         ChatBox.insert(END, "You: " + msg + '\n\n')
#         ChatBox.config(foreground="#446665", font=("Verdana", 12 ))
#         ints = predict_class(msg)
#         res = getResponse(ints, intents)
#         ChatBox.insert(END, "Bot: " + res + '\n\n')
#         ChatBox.config(state=DISABLED)
#         ChatBox.yview(END)
# root = Tk()
# root.title("Chatbot")
# root.geometry("400x500")
# root.resizable(width=FALSE, height=FALSE)
# #Create Chat window
# ChatBox = Text(root, bd=0, bg="white", height="8", width="50", font="Arial",)
# ChatBox.config(state=DISABLED)
# #Bind scrollbar to Chat window
# scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
# ChatBox['yscrollcommand'] = scrollbar.set
# #Create Button to send message
# SendButton = Button(root, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
#                     bd=0, bg="#f9a602", activebackground="#3c9d9b",fg='#000000',
#                     command= send )
# #Create the box to enter message
# EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial")
# #EntryBox.bind("<Return>", send)
# #Place all components on the screen
# scrollbar.place(x=376,y=6, height=386)
# ChatBox.place(x=6,y=6, height=386, width=370)
# EntryBox.place(x=128, y=401, height=90, width=265)
# SendButton.place(x=6, y=401, height=90)
# root.mainloop()

In [36]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
# from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
import random

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
intents_file = open('/content/drive/MyDrive/Colab Notebooks/musicme/intents.json').read()
intents = json.loads(intents_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create the training data
training = []
# create empty array for the output
output_empty = [0] * len(classes)

# training set, bag of words for every sentence
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    word_patterns = doc[0]
    # lemmatize each word - create a base word to represent related words
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # create the bag of words array with 1 if the word is found in the current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    # output is '0' for each tag and '1' for the current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle the features and create a NumPy array
random.shuffle(training)
training = np.array(training, dtype=object)  # Specify dtype as object

# create training and testing lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data is created")

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=172, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")



[(['hi'], 'greeting'), (['hello'], 'greeting'), (['whats', 'up'], 'greeting'), (['sup'], 'greeting'), (['is', 'anyone', 'there'], 'greeting'), (['whats', 'good'], 'greeting'), (['hey'], 'greeting'), (['bye'], 'goodbye'), (['cya'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['goodbye'], 'goodbye'), (['im', 'leaving'], 'goodbye'), (['have', 'a', 'good', 'day'], 'goodbye'), (['how', 'old', 'are', 'you'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['thanks'], 'thanks'), (['thank', 'you'], 'thanks'), (['thankyou'], 'thanks'), (['ty'], 'thanks'), (['I', 'owe', 'you', 'one'], 'thanks'), (['whats', 'is', 'your', 'name'], 'name'), (['whats', 'your', 'name'], 'name'), (['whats', 'should', 'I', 'call', 'you'], 'name'), (['how', 'should', 'I', 'address', 'you'], 'name'), (['Yes', 'it', 'does'], 'sky_net_yes'), (['Yeah'], 'sky_net_yes'), (['Haha', 'yep'], 'sky_net_yes'), (['yes'], 'sky_net_yes'), (['Indeed'], 'sky_net_yes'), (['Yup'], 'sky_net_yes'), (['Just', 'like', 'the', 'terminat

In [37]:
####

In [21]:
#########

In [49]:
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import joblib
import pickle

# Load the trained model
model = load_model('chatbot_model.h5')

# Load the tokenizer and classes
with open('words.pkl', 'rb') as file:
    tokenizer = pickle.load(file)
classes = joblib.load('classes.pkl')

# Define a function to clean and tokenize the input text
def clean_and_tokenize(text):
    # Your text cleaning and tokenization logic here
    # Make sure it's consistent with the preprocessing done during training
    # For example, convert to lowercase, remove punctuation, etc.
    # Tokenize the text using the same tokenizer used during training
    return tokenizer.texts_to_sequences([text])

# Define a function to predict the class of a user input
def predict_class(input_text):
    # Clean and tokenize the input text
    tokenized_text = clean_and_tokenize(input_text)
    # Pad the tokenized text to have the same length as the input during training
    padded_text = pad_sequences(tokenized_text, maxlen=max_sequence_length, padding='post')
    # Make the prediction
    predictions = model.predict(padded_text)
    # Get the predicted class
    predicted_class = classes[np.argmax(predictions)]
    # Return the predicted class
    return predicted_class

# Start a conversation loop
while True:
    # Get user input
    user_input = input('You: ')

    # Check for an exit command
    if user_input.lower() == 'exit':
        print('Goodbye!')
        break

    # Predict the class of the user input
    predicted_class = predict_class(user_input)

    # Print the predicted class
    print(f'Predicted Class: {predicted_class}')


You: hi


AttributeError: 'list' object has no attribute 'texts_to_sequences'

In [22]:
#########

In [23]:
##########

In [27]:
!pip install textblob
!pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 9.2 MB/s eta 0:00:00


In [28]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [33]:
import json
import time
from textblob import TextBlob
import random

client_id = '8be5f0367b7a4b54b730b33faa7c1b2c'
client_secret = '0bc78da67fa44fe985c2586aa74d7e1d'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_track_ids(playlist_id):
    music_id_list = []
    playlist = sp.playlist(playlist_id)

    for item in playlist['tracks']['items']:
        music_track = item['track']
        music_id_list.append(music_track['id'])
    return music_id_list

def analyze_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

# Get sentiment from user input
user_input = input('Enter your message: ')
sentiment = analyze_sentiment(user_input)

# You can use the 'sentiment' variable in your recommendation logic

playlist_id = '37i9dQZF1DX9XIFQuFvzM4'
track_ids = get_track_ids(playlist_id)

# Use the sentiment variable in your recommendation logic
# For example, you can adjust the playlist based on positive or negative sentiment

if sentiment > 0:
    # Positive sentiment logic
    print("Positive sentiment detected!")
    # Add your positive sentiment recommendation logic here
else:
    # Negative sentiment logic
    print("Negative sentiment detected!")
    # Add your negative sentiment recommendation logic here

# The rest of your code remains the same
random.shuffle(track_ids)
embed = '<iframe src="https://open.spotify.com/embed/track/{}" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>'.format(track_ids[0])
print(embed)


Enter your message: love
Positive sentiment detected!
<iframe src="https://open.spotify.com/embed/track/6LzjgzpVl0hB3YrBU1b8Z1" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>
